In [2]:
import os
import PIL
import PIL.Image
import numpy as np

In [8]:
!pip install -U insightface
!pip install onnxruntime
from insightface.app import FaceAnalysis

In [10]:
#download from
!wget "https://www.dropbox.com/scl/fi/b9y03enpi01zfb36ckyir/personas.zip?rlkey=19wypqmaps5b5n28bz3gi7xg7&st=r52y68rh&dl=0" -O personas.zip

--2025-05-31 13:58:11--  https://www.dropbox.com/scl/fi/b9y03enpi01zfb36ckyir/personas.zip?rlkey=19wypqmaps5b5n28bz3gi7xg7&st=r52y68rh&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2774b8d7e39fa1e9d701b4a04c.dl.dropboxusercontent.com/cd/0/inline/Cqsr0121wLi0gkTCb-J5scZsQmnPLHHRco1sXq2wHcKK2UB6xazfiHVgZ0RHvWVwiIrO016RD8ohXCU8Px9j4WdGVdCUSzoaxov9kj_oSzH8lcrgNPYQrEzWOqRZ1RyyOW-SmJKorenEF8Otgq6GKn20/file# [following]
--2025-05-31 13:58:11--  https://uc2774b8d7e39fa1e9d701b4a04c.dl.dropboxusercontent.com/cd/0/inline/Cqsr0121wLi0gkTCb-J5scZsQmnPLHHRco1sXq2wHcKK2UB6xazfiHVgZ0RHvWVwiIrO016RD8ohXCU8Px9j4WdGVdCUSzoaxov9kj_oSzH8lcrgNPYQrEzWOqRZ1RyyOW-SmJKorenEF8Otgq6GKn20/file
Resolving uc2774b8d7e39fa1e9d701b4a04c.dl.dropboxusercontent.com (uc2774b8d7e39fa1e9d701b4a04c.dl.dropboxusercontent.com)... 162

In [11]:
!unzip persona.zip

Archive:  persona.zip
replace personas/barack/face_2.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: personas/barack/face_2.jpeg  
  inflating: personas/barack/barack.jpeg  
  inflating: personas/barack/barack_2.jpeg  
  inflating: personas/dicaprio/dicaprio_2.png  
  inflating: personas/dicaprio/dicaprio.jpg  
  inflating: personas/pancho/pancho.jpeg  
  inflating: personas/pancho/pancho2.jpeg  
  inflating: personas/thalia/thalia2.jpg  
  inflating: personas/thalia/thalia1.jpg  
  inflating: personas/paulina/paulina2.png  
  inflating: personas/paulina/paulina1.jpeg  
  inflating: personas/faces.txt      


In [16]:
datapath = '/content/personas'
ffaces = os.path.join(datapath, 'faces.txt')

with open(ffaces) as f:
  for line in f :
    iname, icl = line.split()
    iname = os.path.join(datapath, iname)
    print('{} --> {}'.format(iname, icl))

/content/personas/paulina/paulina2.png --> paulina 
/content/personas/paulina/paulina1.jpeg --> paulina 
/content/personas/dicaprio/dicaprio_2.png --> dicaprio 
/content/personas/dicaprio/dicaprio.jpg --> dicaprio 
/content/personas/barack/barack.jpeg --> barack 
/content/personas/barack/barack_2.jpeg --> barack 
/content/personas/barack/face_2.jpeg --> barack 
/content/personas/pancho/pancho.jpeg --> pancho 
/content/personas/pancho/pancho2.jpeg --> pancho 
/content/personas/thalia/thalia1.jpg --> thalia 
/content/personas/thalia/thalia2.jpg --> thalia 


In [19]:
# Initialize face analysis model
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])  # Use 'CUDAExecutionProvider' for GPU
app.prepare(ctx_id=0)  # ctx_id=-1 for CPU, 0 for GPU

def get_face_embedding(image_path):
    """Extract face embedding from an image"""
    img = PIL.Image.open(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    faces = app.get(np.array(img))
    if len(faces) < 1:
        raise ValueError("No faces detected in the image")
    if len(faces) > 1:
        print("Warning: Multiple faces detected. Using first detected face")

    return faces[0].embedding

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [21]:
embs = []
names = []
compute = True
with open(ffaces)  as f:
    for line in f :
        iname, icl = line.split()
        iname = os.path.join(datapath, iname.strip())
        print(iname)
        names.append(icl)
        if compute :
            emb = get_face_embedding(iname)
            if len(embs) == 0 :
                embs = emb
            else :
                embs = np.vstack([embs, emb])




/content/personas/paulina/paulina2.png
/content/personas/paulina/paulina1.jpeg
/content/personas/dicaprio/dicaprio_2.png
/content/personas/dicaprio/dicaprio.jpg
/content/personas/barack/barack.jpeg
/content/personas/barack/barack_2.jpeg
/content/personas/barack/face_2.jpeg
/content/personas/pancho/pancho.jpeg
/content/personas/pancho/pancho2.jpeg
/content/personas/thalia/thalia1.jpg
/content/personas/thalia/thalia2.jpg
(11, 512)
[-8.60199749e-01 -3.25832903e-01 -5.88276088e-01 -3.00620079e-01
 -8.68814945e-01  1.08419919e+00 -1.78388691e+00  8.59658062e-01
  2.10448369e-01 -1.12748790e+00  1.81425714e+00  1.48629332e+00
  7.14112282e-01  9.10298049e-01 -1.64289880e+00 -1.86040089e-01
 -1.01427805e+00 -5.27941585e-02 -1.02015615e+00  1.68734074e+00
 -8.31181467e-01  7.06269801e-01  1.77542400e+00  8.04062665e-01
 -2.25114688e-01  1.16627622e+00  1.16601133e+00 -9.17855263e-01
  5.21242976e-01 -4.10577714e-01  4.25215483e-01 -1.12431669e+00
 -1.41559947e+00 -6.14801407e-01  6.19383931e-0

In [22]:
print(embs.shape)
print(embs[2,:])

(11, 512)
[-8.60199749e-01 -3.25832903e-01 -5.88276088e-01 -3.00620079e-01
 -8.68814945e-01  1.08419919e+00 -1.78388691e+00  8.59658062e-01
  2.10448369e-01 -1.12748790e+00  1.81425714e+00  1.48629332e+00
  7.14112282e-01  9.10298049e-01 -1.64289880e+00 -1.86040089e-01
 -1.01427805e+00 -5.27941585e-02 -1.02015615e+00  1.68734074e+00
 -8.31181467e-01  7.06269801e-01  1.77542400e+00  8.04062665e-01
 -2.25114688e-01  1.16627622e+00  1.16601133e+00 -9.17855263e-01
  5.21242976e-01 -4.10577714e-01  4.25215483e-01 -1.12431669e+00
 -1.41559947e+00 -6.14801407e-01  6.19383931e-01  1.58839738e+00
  2.08503246e+00 -7.65935630e-02 -1.48317367e-02 -8.30320477e-01
 -1.42843449e+00 -1.27630651e+00  5.09456754e-01  1.19214725e+00
 -1.42661095e+00 -1.20847917e+00 -8.57758641e-01  4.50519145e-01
 -1.76722556e-02  3.51974159e-01 -3.73316914e-01  3.11537683e-01
  3.64473760e-02  1.77025378e+00 -4.81345594e-01 -4.38780367e-01
 -2.07912460e-01  8.78964365e-01 -1.99835092e-01  1.38903046e+00
 -9.30834711e-0

In [23]:
#np.save("face_embs.npy", embs)
# embs = np.load("face_embs.npy")
#similarity search
embs_norm = embs / np.linalg.norm(embs, ord = 2, axis = 1,  keepdims = True)
sim = embs_norm @ np.transpose(embs_norm)
print(sim)


[[ 1.          0.6209115   0.14419974  0.11881724 -0.04392871 -0.04291671
  -0.04624517  0.10602462  0.06048691 -0.00929999  0.11417773]
 [ 0.6209115   1.          0.15393433  0.09273545 -0.08422911 -0.04639705
  -0.01933917  0.11954491  0.07169361  0.02908938  0.01362212]
 [ 0.14419974  0.15393433  1.0000001   0.8280219   0.08730675  0.06230258
   0.10194063  0.01743324 -0.04671398 -0.05980601 -0.07494242]
 [ 0.11881724  0.09273545  0.8280219   1.          0.12986918  0.0396918
   0.13073254  0.01685694 -0.05304242 -0.08220449 -0.07539892]
 [-0.04392871 -0.08422911  0.08730675  0.12986918  1.0000004   0.69845605
   0.70025647  0.07212625  0.02594766  0.06861601  0.05696481]
 [-0.04291671 -0.04639705  0.06230258  0.0396918   0.69845605  1.
   0.7008226  -0.00924187 -0.04149672  0.03509006  0.0218751 ]
 [-0.04624517 -0.01933917  0.10194063  0.13073254  0.70025647  0.7008226
   0.9999997  -0.00899463 -0.05604562  0.05500643  0.0401415 ]
 [ 0.10602462  0.11954491  0.01743324  0.01685694  

In [24]:
idx_sort =  np.argsort(-sim, axis = 1)
print(idx_sort)


[[ 0  1  2  3 10  7  8  9  5  4  6]
 [ 1  0  2  7  3  8  9 10  6  5  4]
 [ 2  3  1  0  6  4  5  7  8  9 10]
 [ 3  2  6  4  0  1  5  7  8 10  9]
 [ 4  6  5  3  2  7  9 10  8  0  1]
 [ 5  6  4  2  3  9 10  7  8  0  1]
 [ 6  5  4  3  2  9 10  7  1  0  8]
 [ 7  8  1  0  4  2  3  9  6  5 10]
 [ 8  7  1  0  4  9 10  5  2  3  6]
 [ 9 10  4  6  5  1  8  7  0  2  3]
 [10  9  0  4  6  5  1  8  7  2  3]]


In [27]:
for  idx, row in enumerate(idx_sort) :
    name_query = names[idx]
    print('Query Name = {}'.format(name_query))
    result = [(names[i], sim[idx, i]) for i in row[1:5] ]
    print(result)


Query Name = paulina
[('paulina', np.float32(0.6209115)), ('dicaprio', np.float32(0.14419974)), ('dicaprio', np.float32(0.11881724)), ('thalia', np.float32(0.11417773))]
Query Name = paulina
[('paulina', np.float32(0.6209115)), ('dicaprio', np.float32(0.15393433)), ('pancho', np.float32(0.11954491)), ('dicaprio', np.float32(0.09273545))]
Query Name = dicaprio
[('dicaprio', np.float32(0.8280219)), ('paulina', np.float32(0.15393433)), ('paulina', np.float32(0.14419974)), ('barack', np.float32(0.10194063))]
Query Name = dicaprio
[('dicaprio', np.float32(0.8280219)), ('barack', np.float32(0.13073254)), ('barack', np.float32(0.12986918)), ('paulina', np.float32(0.11881724))]
Query Name = barack
[('barack', np.float32(0.70025647)), ('barack', np.float32(0.69845605)), ('dicaprio', np.float32(0.12986918)), ('dicaprio', np.float32(0.087306745))]
Query Name = barack
[('barack', np.float32(0.7008226)), ('barack', np.float32(0.69845605)), ('dicaprio', np.float32(0.06230258)), ('dicaprio', np.float